# 📨 Messaging Systems

<img src=images/lego-pieces-basic.jpg>

## 📨 Traditional Systems

- JMS / MSMQ
- Queues in Databases

## ⚡ Redis & Message Brokers

- Redis
- Message Brokers
    - RabbitMQ
    - Kafka
    - MQTT

docker run -d --name my-redis -p 6379:6379 redis

In [ ]:
#r "nuget: StackExchange.Redis"

In [ ]:
using StackExchange.Redis;

var redis = await ConnectionMultiplexer.ConnectAsync("localhost");
var pubsub = redis.GetSubscriber();

pubsub.Subscribe(RedisChannel.Literal("mychannel"), (channel, message) =>
{
    Console.WriteLine($"Received: {message} from {channel}");
});
await pubsub.PublishAsync(RedisChannel.Literal("mychannel"), "Hello, Redis!");

Thread.Sleep(5000);

- If no one is subscribed when a message is published, the message is lost
- If subscribers are active, they receive the message instantly
- If a subscriber joins later, they will not receive past messages

If you need message persistence or delivery to late subscribers, use Redis Streams or a message queue system like __RabbitMQ__ or __Kafka__

__Abstraction Layer__
- MassTransit; https://masstransit.io
- Wolverine; https://wolverinefx.net

# 💡 Designing for Change

## 💡Idiomatic Coding

The essence of famous book "Clean Code: A Handbook of Agile Software Craftsmanship by Robert C. Martin"
- Code is read far more often than it is written—so write it to be readable, maintainable, and expressive

In [ ]:
var person = new {};

if (person == null)
    Console.WriteLine("Person is null");

if (person is null)
    Console.WriteLine("Person is null");

In [ ]:
if (person != null && person.Address != null && person.Address.City != null)
    Console.WriteLine(person.Address.City);

if (person is { Address.City: not null } cityHolder)
 Console.WriteLine(cityHolder.Address.City);

## 💡 The real cost of software isn't building it. It's changing it

- Software should be easy to change, not just work

The essence of famous book "Refactoring: Improving the Design of Existing Code Edition Unstated by Martin Fowler"
- Design software to embrace change, and use refactoring as the continual process that keeps the code aligned with evolving needs without sacrificing clarity or correctness

__The Career__
- I am able to write code that works
- I am able to write good enough code that can go in product
- I am able to show my code is correct and is good enough that can go in product
- I am able to write a code that's easy to change, not just work

__Code Isn’t Just for Machines — It’s for People__

- Machines don't care about design; humans do
- Design helps the next developer (often you in 6 months) understand what to change
- In DDD, where domain complexity is high, readability and structure directly impact our ability to evolve the model

In [ ]:
class OrderService // One has to figure out where code might be, at one place? at multiple place?
{
    void Process() {}
    void Validate() {}
    void Update() {}
    void Cancel() {}
    void Refund() {}
}

class OrderRefundHandler {} // The design/code is telling where it is

__Design Is an Investment in Change__
- You don’t design to “get it right the first time.” You design to change it later with low risk
- Good design buys you the freedom to adapt when business rules change (which they will)
- DDD assumes you’ll learn more about the domain over time—your code must flex with that learning

In [ ]:
void Send(string channel, string message)
{
    switch (channel)
    {
        case "email":
            //SendEmail
            break;
        case "sms":
            //SendSms
            break;
        case "push":
            //SendPush
            break;
        // keeps growing...
        default:
            throw new ArgumentException("Unknown channel");
    }
}

interface INotify { void Send(string message); }

IEnumerable<INotify> notifiers;
foreach(var notify in notifiers) notify.Send("Hello");

__Localize the Impact of Change__
- A good design contains change within boundaries
- DDD uses Bounded Contexts to do this on a large scale
- Even on the micro level, separating domain logic from infrastructure (e.g., don’t put business rules in controller code) supports the same principle

In [ ]:
dynamic db;
void PlaceOrder(Guid customerId, decimal orderAmount)
{
    var customer = db.Customers.Find(customerId);

    if (customer.CurrentBalance + orderAmount > customer.CreditLimit)
        throw new InvalidOperationException("Credit limit exceeded.");

    customer.CurrentBalance += orderAmount;
    db.SaveChanges();
}

class Customer
{
    void PlaceOrder(decimal orderAmount) { /* Credit Limit checks*/ } // may be later we want loyalty, customer discounts
    // the proper place of cabinet
}

__Bad Design Grows Like Mold__
- How a small compromise (like overusing a Utils class) can lead to a brittle system where everything touches everything
- In DDD, we want high cohesion and low coupling so our domain can evolve in isolation from the tech details

## 🧩 Domain Driven Design for Change

- DDD accepts that change is the norm in business
    - It’s not a bug, it’s a feature
- DDD encourages modeling the domain so that the design reflects how the business evolves:
    - Why does good design matter?
    - We don’t model for perfection — we model to evolve
        - The point of DDD isn’t to write fancy models—it’s to make it safe to change your mind about the model
- Emphasis on language alignment make change safer and more isolated
- Bounded Contexts acting as Strategic pattern
    - A Bounded Context tells you where a model starts and ends
    - It lets different teams or subsystems speak their own language without breaking each other’s code
    - It’s not about how you write code—it’s about how you organize and protect meaning

- Boundaries contain risk

__Design for now vs design for change__

# 🧩 Domain Driven Design

## 📚 Revision

1. Entities
- Objects that have a distinct identity (Id) that runs through time and different states
- The identity is more important than its attributes.
- Example: Customer, Order, Product ✅

2. Value Objects
- Immutable, identity-less objects that describe aspects of the domain.
- Compared based on their attribute values, not identity.
- Example: Money, Address, Email ✅

3. Aggregates
- A cluster of domain objects (Entities + Value Objects) with a single Aggregate Root that enforces consistency and invariants.
- Example: Order (root) with OrderItems ✅

4. Repositories
- Provide methods to access and persist Aggregates.
- Act like an in-memory collection interface to the domain model.
- Example: IOrderRepository, ICustomerRepository ✅

5. Domain Services
- Contain domain logic that doesn’t naturally fit within a single Entity or Value Object.
- Stateless and operate on one or more Aggregates.
- Example: PaymentService, CurrencyConversionService ✅

6. Factories
- Encapsulate complex creation logic for Aggregates or Entities.
- Prevent construction leakage into consumers.
- Example: OrderFactory.CreateFromCart(cart) ✅

7. Domain Events
- Represent something meaningful that happened in the domain.
- Help with eventual consistency, side-effects, and integrating bounded contexts.
- Example: OrderPlaced, CustomerRegistered ✅

8. Specifications
- Encapsulate business rules or selection criteria as objects.
- Often used for querying or validation logic.
- Example: EligibleForDiscountSpecification

9. Modules (a.k.a. Packages)
- Logical grouping of domain concepts to maintain high cohesion and low coupling.
- Helps organize the model and prevent anemic domain models.

## 🔑 Entities / Unique Identity

- In the early stages of designing an Entity, focus only on those primary attributes and behaviors that are central to its unique identity, as well as those useful for querying it
- Purposely ignore all other attributes and behaviors until we settle on the primary ones

In [ ]:
// Domain Entity
public class Employee
{
    public int EmployeeNumber { get; }
    public string FirstName { get; }
    public string LastName { get; }
    public string Department { get; }

    public Employee(int employeeNumber, string firstName, string lastName, string department)
    {
        EmployeeNumber = employeeNumber;
        FirstName = firstName;
        LastName = lastName;
        Department = department;
    }

    public string FullName() => $"{FirstName} {LastName}";
}

// Repository Interface
public interface IEmployeeRepository
{
    Employee GetByEmployeeNumber(int employeeNumber);
    IEnumerable<Employee> GetByFirstName(string firstName);
    IEnumerable<Employee> GetByFullName(string firstName, string lastName);
}

__Sources of Unique Identifier__
- The user provides one or more original unique values as input to the application and the application must ensure that they are unique
    - Title
- The application relies on a persistence store, such as a database, to generate a unique identity
    - Dont do select top 1 id from table order by id
    - @@identity and scope_identity() of SQL Server; https://stackoverflow.com/questions/1920558/what-is-the-difference-between-scope-identity-identity-identity-and-ide
- The application internally generates an identity using an algorithm that ensures uniqueness
    - Accounting Document Numbers, Order / Invoice / Payment, Batch - Serial
    - Guid
- Another Bounded Context has already determined the unique identity. It is input or selected by the user from a set of choices
    - Order Number to ship, for Shipment Context

__The story of Guid, Database Identity, ID and Name__ 👈

Users
- UserGuid
- UserIndex
- UserID
- UserName

__What's Next?__

1. ✅ Validate the Ubiquitous Language
- Talk to domain experts and confirm:
    - Is EmployeeNumber really the identity in their world?
    - Are terms like Department, FirstName, LastName used consistently?
    - Would something like FullName, JobTitle, or Role be more important than initially assumed?
- Outcome: You refine your entity based on shared language and remove ambiguities early
2. ✅ Introduce Value Objects Where Appropriate
- Now that you’ve identified primary attributes, you can extract early Value Objects to enforce invariants and encapsulate logic:
- Update Employee to use them. This gives your model stronger expressiveness and validation boundaries

In [ ]:
public record FullName(string FirstName, string LastName)
{
    public override string ToString() => $"{FirstName} {LastName}";
}

public record Department(string Name);

3. ✅ Identify Core Behaviors (Methods)
- Move beyond data — what actions can an Employee perform or have performed on them? For example:
    - AssignToDepartment(Department department)
    - ChangeName(FullName newName)
- This step transitions your entity from a data container to a rich domain model.
4. ✅ Model Invariants and Business Rules
- Discuss and codify rules like:
    - Can an employee change departments freely?
    - Can employee numbers be reused?
    - Are names required to be unique in a department?
- Begin adding basic validation or domain events where appropriate.
5. ✅ Refine Repository Interface
- With more domain understanding, ask:
    - Should repositories only expose lookups by identity?
    - Should querying needs move to a query model/read side (if using CQRS)? Or create separate query services for full-text or filtered search.

In [ ]:
Employee? FindById(EmployeeNumber id);

6. ✅ Define Bounded Contexts (If System Is Larger)
- Start sketching subdomains:
    - HR Management
    - Payroll
    - Recruitment
- Each may have a slightly different view of an Employee. This avoids an anemic, bloated model

# 📚 Resources

In [ ]:
A realistic image cropped to show a stack of six programming books on a wooden office desk. In the background, a modern computer setup with a monitor, keyboard, and subtle office items is softly blurred. The books are stacked from bottom to top as follows:
1. "The Pragmatic Programmer" – dark cover with white title, moderate thickness.
2. "CLR via C# (Developer Reference)" – dark blue/black hardcover, thick reference size.
3. "Windows Runtime via C# (Developer Reference)" – light blue hardcover, slightly thinner.
4. "Clean Code" by Robert C. Martin – solid black paperback, medium thickness.
5. "Refactoring" by Martin Fowler – white/light cover, medium thickness.
6. "Patterns, Principles, and Practices of Domain‑Driven Design" by Scott Millett & Nick Tune – red/gray argyle-style upper section, thick softcover (~800 pages).

Ensure realistic sizes, textures, visible spines with legible titles, lifelike proportions, and natural office lighting.

<img src=images/dalle-books.png height=600>